# Importing necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns





# Basic info on the Data

In [65]:
df = pd.read_csv("C:\\Project\\corona_tested_individuals_ver_00225.csv")
df

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2022-02-17,0,0,0,0,0,חיובי,No,נקבה,Other
1,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
2,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
3,2022-02-17,0,0,0,0,0,שלילי,No,NaN,Other
4,2022-02-17,0,0,0,0,0,שלילי,No,NaN,Other
...,...,...,...,...,...,...,...,...,...,...
9053443,2020-03-11,0,1,0,1,0,שלילי,NaN,NaN,Abroad
9053444,2020-03-11,0,0,0,0,0,שלילי,NaN,NaN,Other
9053445,2020-03-11,0,0,0,0,0,שלילי,NaN,NaN,Other
9053446,2020-03-11,0,0,0,0,0,שלילי,NaN,NaN,Other


In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6344272 entries, 0 to 9038856
Data columns (total 10 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   test_date            object
 1   cough                int64 
 2   fever                int64 
 3   sore_throat          int64 
 4   shortness_of_breath  int64 
 5   head_ache            int64 
 6   corona_result        object
 7   age_60_and_above     int64 
 8   gender               int64 
 9   test_indication      object
dtypes: int64(7), object(3)
memory usage: 532.4+ MB


In [68]:
df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2022-02-17,0,0,0,0,0,חיובי,No,נקבה,Other
1,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
2,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
7,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
8,2022-02-17,0,0,0,0,0,שלילי,Yes,נקבה,Other


In [95]:
df[df['corona_result']==1].describe()


,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender
count,826347.000000,826347.000000,826347.000000,826347.000000,826347.000000,826347.000000,826347.000000
mean,0.138974,0.116986,0.064633,0.016017,0.125158,0.127350,0.483819
std,0.345920,0.321404,0.245877,0.125543,0.330898,0.333364,0.499738
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [96]:
df[df['corona_result']==0].describe()

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender
count,5.381036e+06,5.381036e+06,5.381036e+06,5.381036e+06,5.381036e+06,5.381036e+06,5.381036e+06
mean,3.663960e-02,2.707100e-02,1.474344e-02,2.497474e-03,2.878702e-02,1.308841e-01,4.854719e-01
std,1.878753e-01,1.622904e-01,1.205242e-01,4.991230e-02,1.672075e-01,3.372736e-01,4.997889e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [97]:
df['gender'].value_counts()

0    3265183
1    3079089
Name: gender, dtype: int64

### Now we will clean all the NaN Data

In [ ]:
df.isnull().sum()
df = df.dropna()

In [93]:
df.shape

(6344272, 10)

### We will now change all the string data to int data:
Female will be 0, male will be 1.
<br> Negative will be 0, positive will be 1.
<br> Under age 60 will be 0, Above will be 1

In [70]:
df['gender'] = df['gender'].replace({"זכר": 1, "נקבה": 0})
df['corona_result'] = df['corona_result'].replace({"חיובי": 1, "שלילי": 0})
df['age_60_and_above'] = df['age_60_and_above'].replace({"Yes": 1, "No": 0})


After we cleaned around 3 milions row of the data, we can still see we have alot of data. we will take a sample from the data that feature distributions are preserved.

In [75]:

sample_df = df.sample(int(len(df) * 0.002))
sample_df.shape


(12688, 10)

### info about the sample data

As we can see we have 13% of postive with corona, like we had before

In [55]:
sample_df[sample_df['corona_result']==1].describe()


,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender
count,1643.000000,1643.000000,1643.000000,1643.000000,1643.000000,1643.000000,1643.000000
mean,0.138771,0.110773,0.052952,0.019477,0.114425,0.136336,0.482045
std,0.345812,0.313946,0.224006,0.138235,0.318423,0.343249,0.499830
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [90]:
sample_df

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
8254194,2020-07-05,0,1,0,0,1,0,0,0,Contact with confirmed
759174,2022-01-10,0,0,0,0,0,1,0,1,Abroad
470332,2022-01-18,0,0,0,0,0,0,0,0,Other
8331900,2020-06-30,0,0,0,0,0,0,0,1,Contact with confirmed
7380917,2020-09-10,0,0,0,0,0,0,0,1,Other
...,...,...,...,...,...,...,...,...,...,...
811020,2022-01-08,0,0,0,0,0,0,0,1,Other
7429054,2020-09-08,0,0,0,0,0,0,1,0,Other
7069247,2020-09-22,0,0,0,0,0,0,0,1,Other
6480554,2020-10-29,0,0,0,0,0,0,1,0,Other


In [98]:
sample_df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
8254194,2020-07-05,0,1,0,0,1,0,0,0,Contact with confirmed
759174,2022-01-10,0,0,0,0,0,1,0,1,Abroad
470332,2022-01-18,0,0,0,0,0,0,0,0,Other
8331900,2020-06-30,0,0,0,0,0,0,0,1,Contact with confirmed
7380917,2020-09-10,0,0,0,0,0,0,0,1,Other


In [100]:
sample_df['gender'].value_counts()

0    6511
1    6177
Name: gender, dtype: int64